In [1]:
from itertools import repeat
import pickle5 as pickle
import pandas as pd
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
import rioxarray as rio
import rasterio
import numpy as np
import geopandas as gpd
from multiprocessing import Pool
import os
from rasterio.merge import merge
from itertools import product
from glob import glob
from pathlib import Path


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']

In [3]:
grid = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/Grid_Ekoreg_2022.gpkg").query('kdPIC=='+'\"'+pic_+'\"')

In [4]:
grid.columns

Index(['left', 'top', 'right', 'bottom', 'ID_GRID', 'identify_1', 'layer',
       'path', 'Bentuklahan', 'Kd_BL', 'Kd_Pulau', 'Nm_Pulau', 'Ekoregion1',
       'Kd_Ekoreg', 'kdPIC_old', 'kdPIC', 'geometry'],
      dtype='object')

In [6]:
from google.cloud import storage

In [11]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [14]:
upload_result=glob('ml_output/08_class_result/result/classified_*')

In [17]:
for i in upload_result:
    y_=i.split('/')[-1]
    upload_blob('bps-gcp-bucket',i,'MLST2023/result/'+y_)

File ml_output/08_class_result/result/classified_ID-3129.tif uploaded to MLST2023/result/classified_ID-3129.tif.
File ml_output/08_class_result/result/classified_ID-3130.tif uploaded to MLST2023/result/classified_ID-3130.tif.
File ml_output/08_class_result/result/classified_ID-3044.tif uploaded to MLST2023/result/classified_ID-3044.tif.
File ml_output/08_class_result/result/classified_ID-3128.tif uploaded to MLST2023/result/classified_ID-3128.tif.
File ml_output/08_class_result/result/classified_ID-2999.tif uploaded to MLST2023/result/classified_ID-2999.tif.
File ml_output/08_class_result/result/classified_ID-3087.tif uploaded to MLST2023/result/classified_ID-3087.tif.
File ml_output/08_class_result/result/classified_ID-3000.tif uploaded to MLST2023/result/classified_ID-3000.tif.
File ml_output/08_class_result/result/classified_ID-3043.tif uploaded to MLST2023/result/classified_ID-3043.tif.
File ml_output/08_class_result/result/classified_ID-3086.tif uploaded to MLST2023/result/classif

In [18]:

client = storage.Client()
ls_=list(client.list_blobs('bps-gcp-bucket', prefix='MLST2023/result'))

In [24]:
result_done=[str(i).split('/')[-1].split(',')[0] for i in ls_ if str(i).split('/')[-1].split(',')[0]!='']

In [25]:
def run_data(idgrid):
    tif_name='classified_'+str(idgrid)+'.tif'
    if tif_name not in result_done:
        run='python3 08_Production.py '+str(idgrid)
        !{run}
    else:
        print(idgrid,' is already classified. skip the process.')

In [ ]:
for i in grid.ID_GRID.tolist()[:50]:
    run_data(i)
    upload_result=glob('ml_output/08_class_result/result/classified_*')
    if len(upload_result)>=10:
        for j in upload_result:
            y_=j.split('/')[-1]
            upload_blob('bps-gcp-bucket',j,'MLST2023/result/'+y_)
            os.remove(j)

ID-2999  is already classified. skip the process.
File ml_output/08_class_result/result/classified_ID-3129.tif uploaded to MLST2023/result/classified_ID-3129.tif.
File ml_output/08_class_result/result/classified_ID-3130.tif uploaded to MLST2023/result/classified_ID-3130.tif.
File ml_output/08_class_result/result/classified_ID-3044.tif uploaded to MLST2023/result/classified_ID-3044.tif.
File ml_output/08_class_result/result/classified_ID-3128.tif uploaded to MLST2023/result/classified_ID-3128.tif.
File ml_output/08_class_result/result/classified_ID-2999.tif uploaded to MLST2023/result/classified_ID-2999.tif.
File ml_output/08_class_result/result/classified_ID-3087.tif uploaded to MLST2023/result/classified_ID-3087.tif.
File ml_output/08_class_result/result/classified_ID-3000.tif uploaded to MLST2023/result/classified_ID-3000.tif.
File ml_output/08_class_result/result/classified_ID-3043.tif uploaded to MLST2023/result/classified_ID-3043.tif.
File ml_output/08_class_result/result/classifi

In [ ]:
#with Pool(28) as p:
#    p.map(run_data,grid.ID_GRID.tolist()[:1])